# Benchmark Dataset Generation

In [1]:
num_sobol = 10000
total_trials = num_sobol
num_repeats = 10 # i.e. how many times an individual experiment is repeated
num_all_campaign_trials = total_trials * num_repeats
print(f"Number of trials in all campaigns: {num_all_campaign_trials}")
est_time_per_trial_s = 5
estimated_campaign_time = num_all_campaign_trials * est_time_per_trial_s / 3600
print(f"Estimated campaign time: {estimated_campaign_time:.1f} hours")

Number of trials in all campaigns: 100000
Estimated campaign time: 138.9 hours


In [2]:
from uuid import uuid4
from self_driving_lab_demo import SelfDrivingLabDemoLight, mqtt_observe_sensor_data

session_id = f"benchmark-dev-{str(uuid4())[0:4]}"
print(f"Session ID: {session_id}")
sdl = SelfDrivingLabDemoLight(
    observe_sensor_data_fn=mqtt_observe_sensor_data,
    observe_sensor_data_kwargs=dict(pico_id="test", session_id=session_id),
    autoload=False,
)

Session ID: benchmark-dev-7e98


In [3]:
parameters = [
    {"name": "R", "type": "range", "bounds": [0, 89]},
    {"name": "G", "type": "range", "bounds": [0, 89]},
    {"name": "B", "type": "range", "bounds": [0, 89]},
    {"name": "atime", "type": "range", "bounds": [0, 100]},
    {"name": "astep", "type": "range", "bounds": [0, 999]},
    {
        "name": "gain",
        "type": "choice",
        "is_ordered": True,
        "values": [0.5, 1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0, 512.0],
    },
]


In [4]:
import json
with open("secrets.json", "r") as f:
    secrets = json.load(f)

SEED = secrets["MAIN_SEED"]

In [5]:
from ax.service.ax_client import AxClient
from ax.modelbridge.factory import get_sobol
from random import shuffle


client = AxClient()
client.create_experiment(parameters=parameters)
m = get_sobol(
    client.experiment.search_space, seed=SEED, fallback_to_sample_polytope=True
)
gr = m.gen(n=num_sobol)
sobol_points = [arm.parameters for arm in gr.arms]
sobol_points = sobol_points * num_repeats # stays flat
shuffle(sobol_points)  # operates inplace
sobol_points[0:3]

[INFO 12-10 23:15:58] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 12-10 23:15:58] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter R. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 12-10 23:15:58] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter G. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 12-10 23:15:58] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter B. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 12-10 23:15:58] ax.service.utils.instan

[{'R': 75, 'G': 65, 'B': 18, 'atime': 68, 'astep': 458, 'gain': 16.0},
 {'R': 55, 'G': 88, 'B': 60, 'atime': 65, 'astep': 805, 'gain': 16.0},
 {'R': 81, 'G': 77, 'B': 9, 'atime': 81, 'astep': 386, 'gain': 8.0}]

In [6]:
from time import time, sleep
from numpy.random import default_rng
import ray

ray.shutdown()
ray.init(num_cpus=3)


rng = default_rng()

t0 = time()


@ray.remote
def observe(params, iter=0):
    sleep(rng.random() * 0.1)  # stagger jobs to avoid overloading the MQTT broker
    result = sdl.observe_sensor_data(params)
    elapsed = (time() - t0) / 3600
    print(
        f"elapsed time (h): {elapsed} | percent complete: {iter/num_all_campaign_trials:.3%} | estimated time remaining (h): {elapsed * (1 - (iter+1)/num_all_campaign_trials)/(iter+1)*num_all_campaign_trials}"
    )
    return result


futures = [observe.remote(point, iter=iter) for iter, point in enumerate(sobol_points)]
data = ray.get(futures)


2022-12-10 23:16:09,026	INFO worker.py:1518 -- Started a local Ray instance.
2022-12-10 23:16:23,382	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::observe() (pid=35664, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 620, in ray._raylet.execute_task
  File "c:\Users\sterg\Miniconda3\envs\sdl-demo\lib\site-packages\ray\_private\memory_monitor.py", line 162, in raise_if_low_memory
    raise RayOutOfMemoryError(
ray._private.memory_monitor.RayOutOfMemoryError: More than 95% of the memory on node Dell-G7 is used (15.36 / 15.77 GB). The top 10 memory consumers are:

PID	MEM	COMMAND
16468	0.44GiB	c:\Users\sterg\Miniconda3\envs\sdl-demo\python.exe -m ipykernel_launcher --ip=127.0.0.1 --stdin=9008
35332	0.38GiB	C:\Program Files\Google\Chrome\Application\chrome.exe
33816	0.38GiB	C:\Program Files\Google\Chrome\Application\chrome.exe --type=renderer --enable-chrome-cart --lang=en
19660	0.35GiB	C:\Users\sterg\AppData\Local\Programs\Microsoft VS Co

## Code Graveyard

In [ ]:
# data = []
# for point in tqdm(sobol_points):
#     futures = [observe.remote(point) for _ in range(num_repeats)]
#     data.append(ray.get(futures))

In [ ]:
# def to_iterator(obj_ids):
#     while obj_ids:
#         done, obj_ids = ray.wait(obj_ids)
#         yield ray.get(done[0])

# obj_ids = [observe.remote(params) for params in sobol_points]
# data = [datum for datum in tqdm(to_iterator(obj_ids))]